In [5]:
import h5py    
import numpy as np    
import torch
from torch import nn
import pandas as pd
import numpy as np
import random
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm

In [6]:
with h5py.File('C2H2-C2H2_ncce31.h5', "r") as f:
  print(f["ener"])
  print(f['grid'])
  y = np.array(f["ener"][:])
  X = f['grid']

<HDF5 dataset "ener": shape (3,), type "<f8">
<HDF5 dataset "grid": shape (259728, 12), type "<f8">


In [7]:
with h5py.File('C6H6_mgae109.h5', "r") as f:
  print(f["ener"])
  print(f['grid'])
  y = np.array(f["ener"][:])
  X_raw = np.array(f["grid"][:])

<HDF5 dataset "ener": shape (3,), type "<f8">
<HDF5 dataset "grid": shape (389592, 12), type "<f8">


In [8]:
X = X_raw[:, 4:-1]

In [9]:
train_size = int(X.shape[0] * 0.8)
X_train = X[:train_size]
X_test = X[train_size:]

nconstants = 21
y_train_dist = [0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*np.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1]
# y_test_dist = np.random.normal(0, 1, nconstants)

In [10]:
y_train = np.tile(y_train_dist, [X_train.shape[0],1])
y_test = np.tile(y_train_dist, [X_test.shape[0],1])

In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
# y_scaler = StandardScaler() 
# y_train = y_scaler.fit_transform(y_train[:, None])
# y_test = y_scaler.transform(y_test[:, None]);

In [13]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [14]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
device

device(type='cuda', index=0)

In [15]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        
    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
    def __len__(self):
        return len(self.X)

BATCH_SIZE = 1024

train_set = Dataset(X=X_train, y=y_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                            batch_size=BATCH_SIZE, shuffle=True) # , num_workers=1

test_set = Dataset(X=X_test, y=y_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                            batch_size=BATCH_SIZE) #, num_workers=1

In [16]:
model = nn.Sequential(
    nn.Linear(7, 64),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(64, 128),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(128, nconstants)
    
)

model.load_state_dict(torch.load('first_test.param'))

<All keys matched successfully>

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [20]:
def train_epoch(model, criterion, optimizer, train_loader):
    model.train().to(device)
    progress_bar = tqdm(train_loader)

    # тут учимся
    accumulated_loss_mse = 0
    accumulated_loss_mae = 0

    for X_batch, y_batch in progress_bar:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device) # переехали на гпу
        predictions = model(X_batch) # смотрим че есть
        # print(predictions)
        loss = criterion(predictions, y_batch) # оцениваем масштабы бедствия
        loss.backward() # обновляем градиенты
        optimizer.step() # делаем шаг градиентного спуска 
        optimizer.zero_grad()

        accumulated_loss_mse += ((predictions - y_batch) ** 2).sum()
        accumulated_loss_mae += abs(predictions - y_batch).sum()
        # progress_bar.set_description("Processing %s" % )
    print('RMSE Loss = {:.4f}'.format((accumulated_loss_mse / len(X_train)) ** 0.5))
    print('MAE Loss = {:.4f}'.format((accumulated_loss_mae / len(X_train))))

In [21]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=1000):
    
    for epoch in range(n_epochs):
        train_epoch(model, criterion, optimizer, train_loader)
        # test(model, criterion, test_loader)

In [22]:
train(model, criterion, optimizer, train_dataloader, test_dataloader)


RMSE Loss = 0.0005
MAE Loss = 0.0002



RMSE Loss = 0.0000
MAE Loss = 0.0000



RMSE Loss = 0.0000
MAE Loss = 0.0000



RMSE Loss = 0.0000
MAE Loss = 0.0000



RMSE Loss = 0.0000
MAE Loss = 0.0000


...


RMSE Loss = 0.0000
MAE Loss = 0.0000



RMSE Loss = 0.0001
MAE Loss = 0.0001



RMSE Loss = 0.0002
MAE Loss = 0.0001



RMSE Loss = 0.0001
MAE Loss = 0.0001


KeyboardInterrupt: 

In [23]:
torch.save(model.state_dict(), 'predoptimized.param')